### Problem statement:
Extracting Product Information from Flipkart Website using Web Scraping


### Objective:
Design a web scraping solution to extract essential product information from the Flipkart website. The goal is to retrieve data such as brands, price, original price, description, and URL of various products listed on Flipkart.

### Requirements:

Website: Utilize the Flipkart website (www.flipkart.com) as the data source for product information extraction.
Data Fields: The following fields need to be extracted for each product:
- Brand: The brand name of the product.
- Price: The current selling price of the product.
- Original Price: The original price of the product (if applicable, considering discounts).
- Description: A brief description of the product.
- URL: The URL of the product's page on Flipkart.

### Flipkart Website

### Importing Necessary Libraries

In [1]:
import pandas as pd               
from bs4 import BeautifulSoup     # BeautifulSoup is used to parse the text information
import requests
from selenium import webdriver

### Code Documentation

In [2]:
# Create a WebDriver instance (make sure you have the appropriate driver executable installed)
driver = webdriver.Chrome()

brands = []
price = []
original_price = []
description = []
link = []
original_url = input('Enter what you want: ')

for i in range(1,5):
    def search_url(url):
        template = 'https://www.flipkart.com/search?q={}&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page={}'
        url = url.replace(' ','+')
        return template.format(url,i)

    driver.get(search_url(original_url))

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Brands
    try:
        brand = soup.find_all('div', class_='_4rR01T')
        for a in brand:
            brands.append(a.text.split('(')[0])
    except AttributeError:
        brands = ''

    # Price
    try:
        pr = soup.find_all('div', class_='_30jeq3 _1_WHN1')
        for b in pr:
            price.append(int(b.text.replace('₹', '').replace(',', '')))
    except AttributeError:
        price.append(None)  # Append None for missing values

    # Original Price
    try:
        orp = soup.find_all('div', class_='_3I9_wc _27UcVY')
        for c in orp:
            original_price.append(int(c.text.replace('₹', '').replace(',', '')))
    except AttributeError:
        original_price.append(None)  # Append None for missing values

    # Append missing values for other lists if necessary
    missing_count = len(brands) - len(price)
    if missing_count > 0:
        price.extend([None] * missing_count)

    missing_count = len(brands) - len(original_price)
    if missing_count > 0:
        original_price.extend([None] * missing_count)


    # Description
    try:
        ul = soup.find_all('ul', class_='_1xgFaf')
        for d in ul:
            li = d.find_all('li', class_='rgWa7D')
            description.append([j.text for j in li])
    except AttributeError:
        description = ''

    # Website link
    try:
        a_tag = soup.find_all('a', class_='_1fQZEK')
        for i in a_tag:
            link.append('https://www.flipkart.com' + i.get('href'))
    except AttributeError:
        link = ''
        
        
# Data Frame
df = pd.DataFrame()
df['brands'] = brands
df['price'] = price
df['original_price'] = original_price
df['description'] = description
df['link'] = link
df.head()

# Close the WebDriver instance
driver.quit()

Enter what you want: mobile


In [3]:
df.head()

,brands,price,original_price,description,link
0,POCO M6 Pro 5G,12999,16999,"[6 GB RAM | 128 GB ROM | Expandable Upto 1 TB,...",https://www.flipkart.com/poco-m6-pro-5g-power-...
1,POCO M6 Pro 5G,10999,14999,"[4 GB RAM | 64 GB ROM | Expandable Upto 1 TB, ...",https://www.flipkart.com/poco-m6-pro-5g-forest...
2,POCO M6 Pro 5G,10999,14999,"[4 GB RAM | 64 GB ROM | Expandable Upto 1 TB, ...",https://www.flipkart.com/poco-m6-pro-5g-power-...
3,POCO C51,6499,9999,"[4 GB RAM | 64 GB ROM | Expandable Upto 1 TB, ...",https://www.flipkart.com/poco-c51-power-black-...
4,POCO C50,5499,8999,[2 GB RAM | 32 GB ROM | Expandable Upto 512 GB...,https://www.flipkart.com/poco-c50-royal-blue-3...


In [4]:
df.to_csv('smart phone.csv')